In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv("C:\\Users\\DELL\\allreviews_samsung.csv")
df = df.drop_duplicates()
df = df.rename(columns ={"comment": "text"})                             
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4929 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   stars        4929 non-null   float64
 1   text         4929 non-null   object 
 2   title        4929 non-null   object 
 3   review_date  4929 non-null   object 
dtypes: float64(1), object(3)
memory usage: 192.5+ KB


In [3]:
df['token_length'] = [len(x.split(" ")) for x in df.text]
max(df.token_length)

296

In [4]:
df.loc[df.token_length.idxmax(),'text']

'\n\n\n\n\n\n\n\n\n\n  \n  \n    \n  Capacity and working was goodThe packaging was really good even for a memory card which one can easily misunderstand for a mobile or similar stuff visuall.However there are a lot of fake ones available so in order to find out which one is real and which is fake check the sides of the memory card.The real ones edges will be white in colour while the fake ones have black coloured edges.Fast data transferring Adapter is include with this card.Pros:-1- The read/write speed of the card through the adapter is around 50/25 mbps.2- The read/write speed of the card through my mobile ZenFone 2 ZE550ML variant with the original cable is around 40/28 mbps well technically it is more than what we get through the original adapter.3- The card really is water proof since I accidentally dropped a glass of water on it and it was still working fine without any problems.4-This card is really fast so fast that I have like 40 movies on my mobile(literally and I am not jo

Data Cleaning

In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [6]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load("en_core_web_sm")

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [7]:
df['clean_text'] = [spacy_cleaner(t) for t in df.text]

In [8]:
#define sentiment class
# decide sentiment as positive, negative and neutral 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [9]:
def sentiment_analyzer_scores(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    if(score['compound']>=0.2):
        return 'pos'
    elif(score['compound']<=-0.2):
        return 'neg'
    else:
        return 'neu'
  

df['sentiment'] = df['clean_text'].apply(lambda x: sentiment_analyzer_scores(x))   

df.sentiment.value_counts()

pos    3766
neu     788
neg     375
Name: sentiment, dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, min_df=5, max_df=0.7, stop_words=None)
X = vectorizer.fit_transform(df.clean_text).toarray()
#from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df.sentiment, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3943, 832) (3943,)
(986, 832) (986,)


In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
sgnb = GaussianNB()
spred_gnb = sgnb.fit(X_train,y_train).predict(X_test)
# summarize the fit of the model
from sklearn import metrics
print(metrics.classification_report(y_test, spred_gnb))
print(metrics.confusion_matrix(y_test, spred_gnb))
print(" accuracy = ", accuracy_score(y_test, spred_gnb))

              precision    recall  f1-score   support

         neg       0.09      0.56      0.16        70
         neu       0.21      0.37      0.27       170
         pos       0.80      0.28      0.42       746

    accuracy                           0.32       986
   macro avg       0.37      0.40      0.28       986
weighted avg       0.65      0.32      0.37       986

[[ 39  10  21]
 [ 75  63  32]
 [314 222 210]]
 accuracy =  0.31643002028397565


In [13]:
################                   dealing with imbalanced data             ###################
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train,np.array(y_train).ravel())

###  building NB model after applying SMOTE 
smote_gnb = sgnb.fit(X_train_res,y_train_res).predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, smote_gnb))
print(metrics.confusion_matrix(y_test, smote_gnb))
print(" accuracy = ", accuracy_score(y_test, smote_gnb))

              precision    recall  f1-score   support

         neg       0.10      0.56      0.16        70
         neu       0.26      0.42      0.32       170
         pos       0.82      0.34      0.48       746

    accuracy                           0.37       986
   macro avg       0.39      0.44      0.32       986
weighted avg       0.67      0.37      0.43       986

[[ 39  10  21]
 [ 63  71  36]
 [302 191 253]]
 accuracy =  0.36815415821501013


In [14]:
>>> from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=0)
rf_pred = rf.fit(X_train, y_train).predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, rf_pred))
print(metrics.confusion_matrix(y_test, rf_pred))
print(" accuracy = ", accuracy_score(y_test, rf_pred))

              precision    recall  f1-score   support

         neg       0.85      0.47      0.61        70
         neu       0.76      0.49      0.60       170
         pos       0.88      0.98      0.93       746

    accuracy                           0.86       986
   macro avg       0.83      0.65      0.71       986
weighted avg       0.85      0.86      0.85       986

[[ 33  14  23]
 [  5  84  81]
 [  1  13 732]]
 accuracy =  0.8610547667342799


In [15]:
################                   dealing with imbalanced data             ###################
###  building RF model after applying SMOTE 
smote_rf = rf.fit(X_train_res,y_train_res).predict(X_test)
# summarize the fit of the model
from sklearn import metrics
print(metrics.classification_report(y_test, smote_rf))
print(metrics.confusion_matrix(y_test, smote_rf))
print(" accuracy = ", accuracy_score(y_test, smote_rf))

              precision    recall  f1-score   support

         neg       0.43      0.56      0.48        70
         neu       0.65      0.64      0.65       170
         pos       0.93      0.91      0.92       746

    accuracy                           0.84       986
   macro avg       0.67      0.70      0.68       986
weighted avg       0.85      0.84      0.84       986

[[ 39  21  10]
 [ 23 109  38]
 [ 29  37 680]]
 accuracy =  0.8397565922920892


In [16]:
# Classifier - Algorithm - SVM
from sklearn import svm
from sklearn.svm import SVC
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM_pred = SVM.fit(X_train, y_train).predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, SVM_pred))
print(metrics.confusion_matrix(y_test, SVM_pred))
print(" accuracy = ", accuracy_score(y_test, SVM_pred))

              precision    recall  f1-score   support

         neg       0.63      0.51      0.57        70
         neu       0.72      0.65      0.68       170
         pos       0.93      0.96      0.94       746

    accuracy                           0.88       986
   macro avg       0.76      0.71      0.73       986
weighted avg       0.87      0.88      0.87       986

[[ 36  19  15]
 [ 18 111  41]
 [  3  25 718]]
 accuracy =  0.8772819472616633


In [17]:
################                   dealing with imbalanced data             ###################
###  building svm model after applying SMOTE 
smote_svm = SVM.fit(X_train_res,y_train_res).predict(X_test)
# summarize the fit of the model
from sklearn import metrics
print(metrics.classification_report(y_test, smote_svm))
print(metrics.confusion_matrix(y_test, smote_svm))
print(" accuracy = ", accuracy_score(y_test, smote_svm))

              precision    recall  f1-score   support

         neg       0.44      0.71      0.54        70
         neu       0.68      0.72      0.70       170
         pos       0.98      0.90      0.94       746

    accuracy                           0.86       986
   macro avg       0.70      0.78      0.73       986
weighted avg       0.89      0.86      0.87       986

[[ 50  16   4]
 [ 34 123  13]
 [ 30  41 675]]
 accuracy =  0.8600405679513184


In [18]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(solver="lbfgs")
# fit the training dataset on the classifier
lr_pred = lr.fit(X_train, y_train).predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, lr_pred))
print(metrics.confusion_matrix(y_test, lr_pred))
print(" accuracy = ", accuracy_score(y_test, lr_pred))

              precision    recall  f1-score   support

         neg       0.62      0.34      0.44        70
         neu       0.66      0.54      0.59       170
         pos       0.89      0.97      0.93       746

    accuracy                           0.85       986
   macro avg       0.72      0.62      0.65       986
weighted avg       0.83      0.85      0.84       986

[[ 24  24  22]
 [ 13  92  65]
 [  2  24 720]]
 accuracy =  0.847870182555781


In [19]:
################                   dealing with imbalanced data             ###################
###  building lr model after applying SMOTE 
smote_lr = lr.fit(X_train_res,y_train_res).predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(y_test, smote_lr))
print(metrics.confusion_matrix(y_test, smote_lr))
print(" accuracy = ", accuracy_score(y_test, smote_lr))

              precision    recall  f1-score   support

         neg       0.39      0.70      0.50        70
         neu       0.65      0.67      0.66       170
         pos       0.98      0.89      0.93       746

    accuracy                           0.84       986
   macro avg       0.67      0.75      0.70       986
weighted avg       0.88      0.84      0.86       986

[[ 49  18   3]
 [ 42 114  14]
 [ 36  43 667]]
 accuracy =  0.8417849898580122


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
